In [10]:
import os
import glob
import gzip
import pandas as pd
import re

data_dir = "/Users/saifahmed/development/ESCC/data/raw/GSE53622_RAW"

# list all txt.gz files
files = sorted(glob.glob(os.path.join(data_dir, "*.txt.gz")))
print(f"Found {len(files)} files")

# show first few
files[:5]

Found 120 files


['/Users/saifahmed/development/ESCC/data/raw/GSE53622_RAW/GSM1296956_US10313827_253831410081_S01_GE2_107_Sep09_1_1a.txt.gz',
 '/Users/saifahmed/development/ESCC/data/raw/GSE53622_RAW/GSM1296957_US10313827_253831410081_S01_GE2_107_Sep09_1_1b.txt.gz',
 '/Users/saifahmed/development/ESCC/data/raw/GSE53622_RAW/GSM1296958_US10313827_253831410081_S01_GE2_107_Sep09_1_2a.txt.gz',
 '/Users/saifahmed/development/ESCC/data/raw/GSE53622_RAW/GSM1296959_US10313827_253831410081_S01_GE2_107_Sep09_1_2b.txt.gz',
 '/Users/saifahmed/development/ESCC/data/raw/GSE53622_RAW/GSM1296960_US10313827_253831410081_S01_GE2_107_Sep09_1_3a.txt.gz']

In [11]:
# PEEK INSIDE ONE FILE
sample = files[0]
print(f"Previewing file: {os.path.basename(sample)}\n{'='*60}")

with gzip.open(sample, "rt", encoding="utf-8", errors="ignore") as f:
    for i, line in enumerate(f):
        print(line.rstrip())
        if i > 40:
            break

Previewing file: GSM1296956_US10313827_253831410081_S01_GE2_107_Sep09_1_1a.txt.gz
TYPE	text	text	text	text	integer	float	float	text	text	text	integer	integer	integer	integer	float	float	float	float	float	float	text	text	text	text	text	text	text	text	text	text	text	text	integer	integer	text	integer	integer	integer	float	float	float	float	float	float	float	float	integer	integer	float	integer	float	float	float	float	integer	float	integer	float	integer	text	integer	integer	float	float	integer	float	float	float	float	float	float	float	integer	integer	integer	float	float	integer	float	float	integer	float	integer	integer	float	integer	integer	text	integer	float	float	integer	float	float	float	float	float	float	float	float	integer	integer	float	float	integer	integer	integer	integer	integer	float	integer	integer	float	integer	integer	integer	integer	float	float	integer	integer	integer	integer	integer	integer	integer	float	float	integer	float	integer	integer	float	integer	float	float	integer	int

In [12]:
# FIND WHERE THE DATA STARTS
def find_data_start(filepath):
    with gzip.open(filepath, "rt", encoding="utf-8", errors="ignore") as f:
        for i, line in enumerate(f):
            if line.strip().startswith(("FEATURE", "FeatureNum")):
                return i
    return None

sample = files[0]
start_line = find_data_start(sample)
print(f"Data section starts around line: {start_line}")


Data section starts around line: 9


In [13]:
if start_line is not None:
    df = pd.read_csv(
        sample,
        sep="\t",                  # tab-separated values
        compression="gzip",
        skiprows=start_line,       # skip metadata
        encoding="utf-8",
        on_bad_lines="skip"
    )
    display(df.head())
    print("\nColumns detected:\n", list(df.columns)[:15])
else:
    print("❌ Could not locate data section automatically.")


,FEATURES,FeatureNum,Row,Col,chr_coord,SubTypeMask,SubTypeName,Start,Sequence,ProbeUID,...,rProcessedBackground,gProcessedBkngError,rProcessedBkngError,IsUsedBGAdjust,gInterpolatedNegCtrlSub,rInterpolatedNegCtrlSub,gIsInNegCtrlRange,rIsInNegCtrlRange,gIsUsedInMD,rIsUsedInMD
0,DATA,1,1,1,NaN,260,BrightCorner,0,NaN,0,...,27.3009,7.07530,13.0194,0,8020.71000,13574.300000,0,0,0,0
1,DATA,2,1,2,NaN,66,Structural,0,NaN,1,...,27.4730,7.28705,13.0422,0,26.22650,24.735900,0,0,0,0
2,DATA,3,1,3,NaN,66,Structural,0,NaN,1,...,27.4399,7.38358,13.1247,0,16.60460,3.047830,0,0,0,0
3,DATA,4,1,4,NaN,66,Structural,0,NaN,1,...,27.3701,7.28870,13.0511,0,8.10577,0.447294,0,0,0,0
4,DATA,5,1,5,NaN,66,Structural,0,NaN,1,...,27.4865,7.43646,13.0125,0,13.69900,12.945400,0,0,0,0



Columns detected:
 ['FEATURES', 'FeatureNum', 'Row', 'Col', 'chr_coord', 'SubTypeMask', 'SubTypeName', 'Start', 'Sequence', 'ProbeUID', 'ControlType', 'ProbeName', 'GeneName', 'SystematicName', 'Description']
